In [ ]:
%pip install pycaret[full]

In [1]:
%pip install markupsafe==2.0.1

  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
werkzeug 2.2.2 requires MarkupSafe>=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.


# Import
pycaret, markupsafe 순서대로 install 하고, 런타임 재시작 후 import

In [1]:
import pandas as pd
import random
import os
import numpy as np

from pycaret.regression import *
from pycaret.utils import check_metric

import sklearn.metrics as metrics

In [2]:
# Seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) 

# Data Load

In [54]:
train_df = pd.read_csv('C:/Users/gaman/Desktop/LG Aimers/해커톤/data/train.csv')

In [55]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y Feature

In [56]:
test_x = pd.read_csv('C:/Users/gaman/Desktop/LG Aimers/해커톤/data/test.csv').drop(columns=['ID'])

In [57]:
# 임의로 test_y 만들기
test_y = train_y
test_y.loc[39607] = 0
for i in range(0,14):
    test_y.iloc[39607,i] = train_y.iloc[:,i].mean()

train_y = train_df.filter(regex='Y') # Output : Y Feature

# 파생 변수 추가
60번 부터 파생변수

### X_60 : 'X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이' 평균

In [58]:
# X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이
sscrew_train = train_x[['X_19','X_20','X_21','X_22']]
sscrew_test = test_x[['X_19','X_20','X_21','X_22']]

# 평균으로 파생변수 생성
train_x['X_60'] = sscrew_train.mean(axis = 1)
test_x['X_60'] = sscrew_test.mean(axis = 1)

### X_60_1 : 'X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이' 평균의 차

In [59]:
# X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이
sscrew_train = train_x[['X_19','X_20','X_21','X_22']]
sscrew_test = test_x[['X_19','X_20','X_21','X_22']]

# X_60 평균 - 값 으로 파생변수 생성
mean60 = train_x.loc[:,'X_60'].mean()
train_x.loc[:,'X_60_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_60_1']= mean60 - train_x.loc[i,'X_60']
  
# X_60 평균 - 값 으로 파생변수 생성
mean60t = test_x.loc[:,'X_60'].mean()
test_x.loc[:,'X_60_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_60_1']= mean60t - test_x.loc[i,'X_60']

### X_60_2 : 'X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이' 분산

In [60]:
# X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이
sscrew_train = train_x[['X_19','X_20','X_21','X_22']]
sscrew_test = test_x[['X_19','X_20','X_21','X_22']]

# 분산으로 파생변수 생성
train_x['X_60_2'] = sscrew_train.var(axis = 1)
test_x['X_60_2'] = sscrew_test.var(axis = 1)

### X_60_3 : 'X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이' 분산의 차

In [61]:
# X_19 ~ X_22 1,2,3,4 스크류 삽입 깊이
sscrew_train = train_x[['X_19','X_20','X_21','X_22']]
sscrew_test = test_x[['X_19','X_20','X_21','X_22']]

# X_60_2 평균 - 값 으로 파생변수 생성
mean60_2 = train_x.loc[:,'X_60_2'].mean()
train_x.loc[:,'X_60_3'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_60_3']= mean60_2 - train_x.loc[i,'X_60_2']
  
# X_60_2 평균 - 값 으로 파생변수 생성
mean60_2t = test_x.loc[:,'X_60_2'].mean()
test_x.loc[:,'X_60_3'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_60_3']= mean60_2t - test_x.loc[i,'X_60_2']

### X_61 : 'X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량' 평균

In [62]:
# X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량
smt_train = train_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]
smt_test = test_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]

# 평균으로 파생변수 생성
train_x['X_61'] = smt_train.mean(axis = 1)
test_x['X_61'] = smt_test.mean(axis = 1)

### X_61_1 : 'X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량' 평균의 차 

In [63]:
# X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량
smt_train = train_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]
smt_test = test_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]

# X_61 평균 - 값 으로 파생변수 생성
mean61 = train_x.loc[:,'X_61'].mean()
train_x.loc[:,'X_61_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_61_1']= mean61 - train_x.loc[i,'X_61']

# X_61 평균 - 값 으로 파생변수 생성
mean61t = test_x.loc[:,'X_61'].mean()
test_x.loc[:,'X_61_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_61_1']= mean61t - test_x.loc[i,'X_61']

### X_62 : 'X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량' 분산

In [64]:
# X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량
smt_train = train_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]
smt_test = test_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]

# 분산으로 파생변수 생성
train_x['X_62'] = smt_train.var(axis = 1)
test_x['X_62'] = smt_test.var(axis = 1)

### X_63 : 'X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수' 평균

In [65]:
# X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수
pin_train = train_x[['X_24','X_25','X_26','X_27','X_28','X_29']]
pin_test = test_x[['X_24','X_25','X_26','X_27','X_28','X_29']]

# 평균으로 파생변수 생성
train_x['X_63'] = pin_train.mean(axis = 1)
test_x['X_63'] = pin_test.mean(axis = 1)

### X_63_1 : 'X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수' 평균의 차

In [66]:
# X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수
pin_train = train_x[['X_24','X_25','X_26','X_27','X_28','X_29']]
pin_test = test_x[['X_24','X_25','X_26','X_27','X_28','X_29']]

# X_63 평균 - 값 으로 파생변수 생성
mean63 = train_x.loc[:,'X_63'].mean()
train_x.loc[:,'X_63_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_63_1']= mean63 - train_x.loc[i,'X_63']
  
# X_63 평균 - 값 으로 파생변수 생성
mean63t = test_x.loc[:,'X_63'].mean()
test_x.loc[:,'X_63_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_63_1']= mean63t - test_x.loc[i,'X_63']

### X_63_2 : 'X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수' 분산

In [67]:
# X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수
pin_train = train_x[['X_24','X_25','X_26','X_27','X_28','X_29']]
pin_test = test_x[['X_24','X_25','X_26','X_27','X_28','X_29']]

# 평균으로 파생변수 생성
train_x['X_63_2'] = pin_train.var(axis = 1)
test_x['X_63_2'] = pin_test.var(axis = 1)

### X_63_3 : 'X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수' 분산의 차

In [68]:
# X_24 ~ X_29 커넥터 1,2,3,4,5,6번 핀 치수
pin_train = train_x[['X_24','X_25','X_26','X_27','X_28','X_29']]
pin_test = test_x[['X_24','X_25','X_26','X_27','X_28','X_29']]

# X_63_2 평균 - 값 으로 파생변수 생성
mean63_2 = train_x.loc[:,'X_63_2'].mean()
train_x.loc[:,'X_63_3'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_63_3']= mean63_2 - train_x.loc[i,'X_63_2']
  
# X_63_2 평균 - 값 으로 파생변수 생성
mean63_2t = test_x.loc[:,'X_63_2'].mean()
test_x.loc[:,'X_63_3'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_63_3']= mean63_2t - test_x.loc[i,'X_63_2']

### X_64 : 'X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량' 분산의 차

In [69]:
# X_50 ~ X_56 RF1,2,3,4,5,6,7 부분 SMT 납 량
smt_train = train_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]
smt_test = test_x[['X_50','X_51','X_52','X_53','X_54','X_55','X_56']]

# X_62 평균 - 값 으로 파생변수 생성
mean62 = train_x.loc[:,'X_62'].mean()
train_x.loc[:,'X_64'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_64']= mean62 - train_x.loc[i,'X_62']
  
# X_62 평균 - 값 으로 파생변수 생성
mean62t = test_x.loc[:,'X_62'].mean()
test_x.loc[:,'X_64'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_64']= mean62t - test_x.loc[i,'X_62']

### X_65 : 'X_14 ~ X_18' 안테나 패드 위치의 평균

In [70]:
# X_14 ~ X_18 안테나 패드 위치 1,2,3,4,5의 평균
an_train = train_x[['X_14','X_15','X_16','X_17','X_18']]
an_test = test_x[['X_14','X_15','X_16','X_17','X_18']]

# 평균으로 파생변수 생성
train_x['X_65'] = an_train.mean(axis = 1)
test_x['X_65'] = an_test.mean(axis = 1)

### X_66 : 'X_14 ~ X_18' 안테나 패드 위치의 분산

In [71]:
# X_14 ~ X_18 안테나 패드 위치 1,2,3,4,5의 분산
an_train = train_x[['X_14','X_15','X_16','X_17','X_18']]
an_test = test_x[['X_14','X_15','X_16','X_17','X_18']]

# 분산으로 파생변수 생성
train_x['X_66'] = an_train.var(axis = 1)
test_x['X_66'] = an_test.var(axis = 1)

### X_67 : 'X_14 ~ X_18' 안테나 패드 위치의 분산 차이

In [72]:
# X_14 ~ X_18 안테나 패드 위치 1,2,3,4,5의 분산
an_train = train_x[['X_14','X_15','X_16','X_17','X_18']]
an_test = test_x[['X_14','X_15','X_16','X_17','X_18']]

# X_66 평균 - 값 으로 파생변수 생성
mean66 = train_x.loc[:,'X_66'].mean()
train_x.loc[:,'X_67'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_67']= mean66 - train_x.loc[i,'X_66']
  
# X_66 평균 - 값 으로 파생변수 생성
mean66t = test_x.loc[:,'X_66'].mean()
test_x.loc[:,'X_67'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_67']= mean66t - test_x.loc[i,'X_66']

### X_67_1 : 'X_14 ~ X_18' 안테나 패드 위치의 평균 차이

In [73]:
# X_14 ~ X_18 안테나 패드 위치 1,2,3,4,5의 평균
an_train = train_x[['X_14','X_15','X_16','X_17','X_18']]
an_test = test_x[['X_14','X_15','X_16','X_17','X_18']]

# X_65 평균 - 값 으로 파생변수 생성
mean65 = train_x.loc[:,'X_65'].mean()
train_x.loc[:,'X_67_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_67_1']= mean65 - train_x.loc[i,'X_65']
  
# X_65 평균 - 값 으로 파생변수 생성
mean65t = test_x.loc[:,'X_65'].mean()
test_x.loc[:,'X_67_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_67_1']= mean65t - test_x.loc[i,'X_65']

### X_68 : ‘X_07 ~ X_09’ 방열 재료 면적의 평균

In [74]:
# X_07 ~ X_09 방열 재료 면적 1,2,3의 평균
hitw_train = train_x[['X_07','X_08','X_09']]
hitw_test = test_x[['X_07','X_08','X_09']]

# 평균으로 파생변수 생성
train_x['X_68'] = hitw_train.mean(axis = 1)
test_x['X_68'] = hitw_test.mean(axis = 1)

### X_68_1 : ‘X_07 ~ X_09’ 방열 재료 면적의 분산

In [75]:
# X_07 ~ X_09 방열 재료 면적 1,2,3의 분산
hitw_train = train_x[['X_07','X_08','X_09']]
hitw_test = test_x[['X_07','X_08','X_09']]

# 분산으로 파생변수 생성
train_x['X_68_1'] = hitw_train.var(axis = 1)
test_x['X_68_1'] = hitw_test.var(axis = 1)

### X_69 : 'X_03, X_10, X_11' 방열 재료 무게의 평균

In [76]:
# X_03, X_10, X_11 방열 재료 무게 1,2,3
hitg_train = train_x[['X_03','X_10','X_11']]
hitg_test = test_x[['X_03','X_10','X_11']]

# 평균으로 파생변수 생성
train_x['X_69'] = hitg_train.mean(axis = 1)
test_x['X_69'] = hitg_test.mean(axis = 1)

### X_69_1 : 'X_03, X_10, X_11' 방열 재료 무게의 분산

In [77]:
# X_03, X_10, X_11 방열 재료 무게 1,2,3
hitg_train = train_x[['X_03','X_10','X_11']]
hitg_test = test_x[['X_03','X_10','X_11']]

# 평균으로 파생변수 생성
train_x['X_69_1'] = hitg_train.var(axis = 1)
test_x['X_69_1'] = hitg_test.var(axis = 1)

### X_70 : ‘X_07 ~ X_09’ 방열 재료 면적의 평균차

In [78]:
# X_07 ~ X_09 방열 재료 면적 1,2,3
hitw_train = train_x[['X_07','X_08','X_09']]
hitw_test = test_x[['X_07','X_08','X_09']]

# X_68 평균 - 값 으로 파생변수 생성
mean68 = train_x.loc[:,'X_68'].mean()
train_x.loc[:,'X_70'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_70']= mean68 - train_x.loc[i,'X_68']
  
# X_68 평균 - 값 으로 파생변수 생성
mean68t = test_x.loc[:,'X_68'].mean()
test_x.loc[:,'X_70'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_70']= mean68t - test_x.loc[i,'X_68']

### X_70_1 : ‘X_07 ~ X_09’ 방열 재료 면적의 분산 차

In [79]:
# X_07 ~ X_09 방열 재료 면적 1,2,3
hitw_train = train_x[['X_07','X_08','X_09']]
hitw_test = test_x[['X_07','X_08','X_09']]

# X_68_1 평균 - 값 으로 파생변수 생성
mean68_1 = train_x.loc[:,'X_68_1'].mean()
train_x.loc[:,'X_70_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_70_1']= mean68_1 - train_x.loc[i,'X_68_1']
  
# X_68_1 평균 - 값 으로 파생변수 생성
mean68_1t = test_x.loc[:,'X_68_1'].mean()
test_x.loc[:,'X_70_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_70_1']= mean68_1t - test_x.loc[i,'X_68_1']

### X_71 : 'X_03, X_10, X_11' 방열 재료 무게의 평균차

In [80]:
# X_03, X_10, X_11 방열 재료 무게 1,2,3
hitg_train = train_x[['X_03','X_10','X_11']]
hitg_test = test_x[['X_03','X_10','X_11']]

# X_69 평균 - 값 으로 파생변수 생성
mean69 = train_x.loc[:,'X_69'].mean()
train_x.loc[:,'X_71'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_71']= mean69 - train_x.loc[i,'X_69']
  
# X_69 평균 - 값 으로 파생변수 생성
mean69t = test_x.loc[:,'X_69'].mean()
test_x.loc[:,'X_71'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_71']= mean69t - test_x.loc[i,'X_69']

### X_71_1 : 'X_03, X_10, X_11' 방열 재료 무게의 분산 차

In [81]:
# X_03, X_10, X_11 방열 재료 무게 1,2,3
hitg_train = train_x[['X_03','X_10','X_11']]
hitg_test = test_x[['X_03','X_10','X_11']]

# X_69_1 평균 - 값 으로 파생변수 생성
mean69_1 = train_x.loc[:,'X_69_1'].mean()
train_x.loc[:,'X_71_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_71_1']= mean69_1 - train_x.loc[i,'X_69_1']
  
# X_69_1 평균 - 값 으로 파생변수 생성
mean69_1t = test_x.loc[:,'X_69_1'].mean()
test_x.loc[:,'X_71_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_71_1']= mean69_1t - test_x.loc[i,'X_69_1']

### X_72 : 'X_30 ~ X_33' 스크류 삽입 깊이 1,2,3,4 평균

In [82]:
# X_30 ~ X_33 스크류 삽입 깊이 1,2,3,4
sscrew2_train = train_x[['X_30','X_31','X_32','X_33']]
sscrew2_test = test_x[['X_30','X_31','X_32','X_33']]

# 평균으로 파생변수 생성
train_x['X_72'] = sscrew2_train.mean(axis = 1)
test_x['X_72'] = sscrew2_test.mean(axis = 1)

### X_72_1 : 'X_30 ~ X_33' 스크류 삽입 깊이 1,2,3,4 분산

In [83]:
# X_30 ~ X_33 스크류 삽입 깊이 1,2,3,4
sscrew2_train = train_x[['X_30','X_31','X_32','X_33']]
sscrew2_test = test_x[['X_30','X_31','X_32','X_33']]

# 분산으로 파생변수 생성
train_x['X_72_1'] = sscrew2_train.var(axis = 1)
test_x['X_72_1'] = sscrew2_test.var(axis = 1)

### X_73 : 'X_30 ~ X_33' 스크류 삽입 깊이 1,2,3,4 평균의 차

In [84]:
# X_30 ~ X_33 스크류 삽입 깊이 1,2,3,4
sscrew2_train = train_x[['X_30','X_31','X_32','X_33']]
sscrew2_test = test_x[['X_30','X_31','X_32','X_33']]

# X_72 평균 - 값 으로 파생변수 생성
mean72 = train_x.loc[:,'X_72'].mean()
train_x.loc[:,'X_73'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_73']= mean72 - train_x.loc[i,'X_72']
  
# X_72 평균 - 값 으로 파생변수 생성
mean72t = test_x.loc[:,'X_72'].mean()
test_x.loc[:,'X_73'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_73']= mean72t - test_x.loc[i,'X_72']

### X_73_1 : 'X_30 ~ X_33' 스크류 삽입 깊이 1,2,3,4 분산의 차

In [85]:
# X_30 ~ X_33 스크류 삽입 깊이 1,2,3,4
sscrew2_train = train_x[['X_30','X_31','X_32','X_33']]
sscrew2_test = test_x[['X_30','X_31','X_32','X_33']]

# X_72_1 평균 - 값 으로 파생변수 생성
mean72_1 = train_x.loc[:,'X_72_1'].mean()
train_x.loc[:,'X_73_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_73_1']= mean72_1 - train_x.loc[i,'X_72_1']
  
# X_72_1 평균 - 값 으로 파생변수 생성
mean72_1t = test_x.loc[:,'X_72_1'].mean()
test_x.loc[:,'X_73_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_73_1']= mean72_1t - test_x.loc[i,'X_72_1']

### X_74 : 'X_41 ~ X_44 레이돔 치수 1,2,3,4' 분산

In [86]:
# X_41 ~ X_44 스크류 삽입 깊이 1,2,3,4 
ray_train = train_x[['X_41','X_42','X_43','X_44']]
ray_test = test_x[['X_41','X_42','X_43','X_44']]

# 분산으로 파생변수 생성
train_x['X_74'] = ray_train.var(axis = 1)
test_x['X_74'] = ray_test.var(axis = 1)

### X_74_1 : 'X_41 ~ X_44 레이돔 치수 1,2,3,4' 평균

In [87]:
# X_41 ~ X_44 스크류 삽입 깊이 1,2,3,4 
ray_train = train_x[['X_41','X_42','X_43','X_44']]
ray_test = test_x[['X_41','X_42','X_43','X_44']]

# 평균으로 파생변수 생성
train_x['X_74_1'] = ray_train.mean(axis = 1)
test_x['X_74_1'] = ray_test.mean(axis = 1)

### X_75 : 'X_41 ~ X_44 레이돔 치수 1,2,3,4' 분산의 차

In [88]:
# X_74 평균 - 값 으로 파생변수 생성
mean74 = train_x.loc[:,'X_74'].mean()
train_x.loc[:,'X_75'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_75']= mean74 - train_x.loc[i,'X_74']
  
# X_74 평균 - 값 으로 파생변수 생성
mean74t = test_x.loc[:,'X_74'].mean()
test_x.loc[:,'X_75'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_75']= mean74t - test_x.loc[i,'X_74']

### X_75_1 : 'X_41 ~ X_44 레이돔 치수 1,2,3,4' 평균의 차

In [89]:
# X_74_1 평균 - 값 으로 파생변수 생성
mean74_1 = train_x.loc[:,'X_74_1'].mean()
train_x.loc[:,'X_75_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_75_1']= mean74_1 - train_x.loc[i,'X_74_1']
  
# X_74_1 평균 - 값 으로 파생변수 생성
mean74_1t = test_x.loc[:,'X_74_1'].mean()
test_x.loc[:,'X_75_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_75_1']= mean74_1t - test_x.loc[i,'X_74_1']

### X_76 : 'X_34 ~ X_37' 스크류 체결 시 분당 회전수 1,2,3,4 평균

In [90]:
# 'X_34 ~ X_37' 스크류 체결 시 분당 회전수 1,2,3,4 
turn_train = train_x[['X_34','X_35','X_36','X_37']]
turn_test = test_x[['X_34','X_35','X_36','X_37']]

# 평균으로 파생변수 생성
train_x['X_76'] = turn_train.mean(axis = 1)
test_x['X_76'] = turn_test.mean(axis = 1)

### X_76_1 : 'X_34 ~ X_37' 스크류 체결 시 분당 회전수 1,2,3,4 분산

In [91]:
# 'X_34 ~ X_37' 스크류 체결 시 분당 회전수 1,2,3,4 
turn_train = train_x[['X_34','X_35','X_36','X_37']]
turn_test = test_x[['X_34','X_35','X_36','X_37']]

# 분산으로 파생변수 생성
train_x['X_76_1'] = turn_train.var(axis = 1)
test_x['X_76_1'] = turn_test.var(axis = 1)

### X_77 : 'X_34 ~ X_37' 스크류 체결 시 분당 회전수 1,2,3,4 평균의 차이

In [92]:
# X_76 평균 - 값 으로 파생변수 생성
mean76 = train_x.loc[:,'X_76'].mean()
train_x.loc[:,'X_77'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_77']= mean76 - train_x.loc[i,'X_76']
  
# X_76 평균 - 값 으로 파생변수 생성
mean76t = test_x.loc[:,'X_76'].mean()
test_x.loc[:,'X_77'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_77']= mean76t - test_x.loc[i,'X_76']

### X_77_1 : 'X_34 ~ X_37' 스크류 체결 시 분당 회전수 1,2,3,4 분산의 차이

In [93]:
# X_76_1 평균 - 값 으로 파생변수 생성
mean76_1 = train_x.loc[:,'X_76_1'].mean()
train_x.loc[:,'X_77_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_77_1']= mean76_1 - train_x.loc[i,'X_76_1']
  
# X_76_1 평균 - 값 으로 파생변수 생성
mean76_1t = test_x.loc[:,'X_76_1'].mean()
test_x.loc[:,'X_77_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_77_1']= mean76_1t - test_x.loc[i,'X_76_1']

### X_78 : 'X_38 ~ X_40' 하우징 PCB 안착부 1,2,3 치수 평균

In [94]:
# 'X_38 ~ X_40' 하우징 PCB 안착부 1,2,3 치수
husing_train = train_x[['X_38','X_39','X_40']]
husing_test = test_x[['X_38','X_39','X_40']]

# 평균으로 파생변수 생성
train_x['X_78'] = husing_train.mean(axis = 1)
test_x['X_78'] = husing_test.mean(axis = 1)

### X_78_1 : 'X_38 ~ X_40' 하우징 PCB 안착부 1,2,3 치수 분산

In [95]:
# 'X_38 ~ X_40' 하우징 PCB 안착부 1,2,3 치수
husing_train = train_x[['X_38','X_39','X_40']]
husing_test = test_x[['X_38','X_39','X_40']]

# 분산으로 파생변수 생성
train_x['X_78_1'] = husing_train.var(axis = 1)
test_x['X_78_1'] = husing_test.var(axis = 1)

### X_79 : 'X_38 ~ X_40' 하우징 PCB 안착부 1,2,3 치수 평균의 차

In [96]:
# X_78 평균 - 값 으로 파생변수 생성
mean78 = train_x.loc[:,'X_78'].mean()
train_x.loc[:,'X_79'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_79']= mean78 - train_x.loc[i,'X_78']
  
# X_78 평균 - 값 으로 파생변수 생성
mean78t = test_x.loc[:,'X_78'].mean()
test_x.loc[:,'X_79'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_79']= mean78t - test_x.loc[i,'X_78']

### X_79_1 : 'X_38 ~ X_40' 하우징 PCB 안착부 1,2,3 치수 분산의 차

In [97]:
# X_78_1 평균 - 값 으로 파생변수 생성
mean78_1 = train_x.loc[:,'X_78_1'].mean()
train_x.loc[:,'X_79_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_79_1']= mean78_1 - train_x.loc[i,'X_78_1']
  
# X_78_1 평균 - 값 으로 파생변수 생성
mean78_1t = test_x.loc[:,'X_78_1'].mean()
test_x.loc[:,'X_79_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_79_1']= mean78_1t - test_x.loc[i,'X_78_1']

### X_80 : 'X_01, X_02, X_05, X_06' PCB 체결 시 단계별 누름 량 의 합

In [98]:
# 'X_01, X_02, X_05, X_06' PCB 체결 시 단계별 누름 량
push_train = train_x[['X_01', 'X_02', 'X_05', 'X_06']]
push_test = test_x[['X_01', 'X_02', 'X_05', 'X_06']]

# 합으로 파생변수 생성
train_x['X_80'] = push_train.sum(axis = 1)
test_x['X_80'] = push_test.sum(axis = 1)

### X_80_1 : 'X_01, X_02, X_05, X_06' PCB 체결 시 단계별 누름 량 의 합의 평균의 차

In [99]:
# X_80 평균 - 값 으로 파생변수 생성
mean80 = train_x.loc[:,'X_80'].mean()
train_x.loc[:,'X_80_1'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_80_1']= mean80 - train_x.loc[i,'X_80']
  
# X_80 평균 - 값 으로 파생변수 생성
mean80t = test_x.loc[:,'X_80'].mean()
test_x.loc[:,'X_80_1'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_80_1']= mean80t - test_x.loc[i,'X_80']

### X_81 : 'X_12' 커넥터 위치 기준 좌표 평균의 차

In [100]:
# X_12 평균 - 값 으로 파생변수 생성
mean12 = train_x.loc[:,'X_12'].mean()
train_x.loc[:,'X_81'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_81']= mean12 - train_x.loc[i,'X_12']
  
# X_12 평균 - 값 으로 파생변수 생성
mean12t = test_x.loc[:,'X_12'].mean()
test_x.loc[:,'X_81'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_81']= mean12t - test_x.loc[i,'X_12']

### X_82 : 'X_13' 각 안테나 패드 위치(높이) 차이 의 평균의 차

In [101]:
# X_13 평균 - 값 으로 파생변수 생성
mean13 = train_x.loc[:,'X_13'].mean()
train_x.loc[:,'X_82'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_82']= mean13 - train_x.loc[i,'X_13']
  
# X_13 평균 - 값 으로 파생변수 생성
mean13t = test_x.loc[:,'X_13'].mean()
test_x.loc[:,'X_82'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_82']= mean13t - test_x.loc[i,'X_13']

### X_83 : 'X_45' 안테나 부분 레이돔 기울기 평균의 차

In [102]:
# X_45 평균 - 값 으로 파생변수 생성
mean45 = train_x.loc[:,'X_45'].mean()
train_x.loc[:,'X_83'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_83']= mean45 - train_x.loc[i,'X_45']
  
# X_45 평균 - 값 으로 파생변수 생성
mean45t = test_x.loc[:,'X_45'].mean()
test_x.loc[:,'X_83'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_83']= mean45t - test_x.loc[i,'X_45']

### X_84 : 'X_46' 실란트 본드 소요량 평균 의 차

In [103]:
# X_46 평균 - 값 으로 파생변수 생성
mean46 = train_x.loc[:,'X_46'].mean()
train_x.loc[:,'X_84'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_84']= mean46 - train_x.loc[i,'X_46']
  
# X_46 평균 - 값 으로 파생변수 생성
mean46t = test_x.loc[:,'X_46'].mean()
test_x.loc[:,'X_84'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_84']= mean46t - test_x.loc[i,'X_46']

### X_85 : 'X_49' Cal 투입 전 대기 시간 평균의 차

In [104]:
# X_49 평균 - 값 으로 파생변수 생성
mean49 = train_x.loc[:,'X_49'].mean()
train_x.loc[:,'X_85'] = 0

for i in range(len(train_x)):
  train_x.loc[i,'X_85']= mean49 - train_x.loc[i,'X_49']
  
# X_49 평균 - 값 으로 파생변수 생성
mean49t = test_x.loc[:,'X_49'].mean()
test_x.loc[:,'X_85'] = 0

for i in range(len(test_x)):
  test_x.loc[i,'X_85']= mean49t - test_x.loc[i,'X_49']

In [108]:
# 제외할 변수
train_x.drop(['X_23','X_47','X_48','X_60','X_60_2','X_61','X_61_1','X_62','X_63_1','X_63_2','X_63_3','X_72_1','X_73_1','X_74','X_78_1'], axis=1) 
test_x.drop(['X_23','X_47','X_48','X_60','X_60_2','X_61','X_61_1','X_62','X_63_1','X_63_2','X_63_3','X_72_1','X_73_1','X_74','X_78_1'], axis=1) 

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_78,X_79,X_79_1,X_80,X_80_1,X_81,X_82,X_83,X_84,X_85
0,68.504,103.321,76.67,1,101.867,73.963,30.51,63.57,239.80,0.0,...,-16.393333,0.266327,0.108766,347.655,-3.016883,0.003444,-0.006603,-0.005415,-0.696147,-567.942293
1,67.485,103.320,69.37,1,101.992,67.845,28.03,116.99,189.23,0.0,...,-15.536667,-0.590339,0.087866,340.642,3.996117,0.013444,-0.026603,-0.115415,6.303853,-474.842293
2,69.524,103.320,68.97,1,101.884,77.022,29.65,205.68,214.93,0.0,...,-16.523333,0.396327,0.120266,351.750,-7.111883,-0.056556,-0.056603,0.014585,-0.696147,1798.857707
3,69.524,103.320,65.87,1,101.866,73.963,28.15,103.38,180.80,0.0,...,-16.313333,0.186327,0.114166,348.673,-4.034883,-0.016556,-0.036603,0.024585,-0.696147,1407.157707
4,73.603,103.321,66.67,1,101.891,74.983,29.92,71.20,231.93,0.0,...,-16.330000,0.202994,0.116700,353.798,-9.159883,-0.006556,0.023397,0.064585,-0.696147,5907.457707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,68.504,103.320,63.97,1,103.157,68.864,29.49,116.35,284.16,0.0,...,-16.436667,0.309661,0.108966,343.845,0.793117,0.023444,0.003397,0.044585,-0.696147,-45463.842293
39604,68.504,103.320,61.37,1,103.137,68.864,32.29,116.28,272.41,0.0,...,-16.313333,0.186327,0.128166,343.825,0.813117,0.033444,-0.016603,-0.035415,-0.696147,-45184.442293
39605,69.524,103.320,63.67,1,103.149,69.884,30.00,113.05,295.54,0.0,...,-16.493333,0.366327,0.113366,345.877,-1.238883,-0.006556,-0.016603,0.034585,-0.696147,-43617.842293
39606,67.485,103.321,61.77,1,103.148,67.845,32.05,115.05,267.26,0.0,...,-16.236667,0.109661,0.116666,341.799,2.839117,0.003444,-0.016603,0.044585,-0.696147,-43577.042293


# Modeling

In [110]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

In [125]:
lightgbm = MultiOutputRegressor(LGBMRegressor(random_state=42))

# params={'learning_rate': [0.005, 0.01, 0.03, 0.05],
#                      'max_depth': [3, 5, 7],
#                      'n_estimators': [500, 1000, 2000, 3000]
#                      }

hyperparameters = dict(estimator__learning_rate=[0.01,0.03, 0.05, 0.005, 0.06], 
                     estimator__n_estimators=[200,300,500, 700,1000],
                     estimator__max_depth=[5, 10,20,30]
                    )

random_search = RandomizedSearchCV(lightgbm, param_distributions=hyperparameters, cv=5, scoring='neg_root_mean_squared_error', random_state=42)
tuning = random_search.fit(train_x, train_y)

KeyboardInterrupt: 

In [120]:
random_search.best_params_

{'estimator__n_estimators': 500,
 'estimator__max_depth': 20,
 'estimator__learning_rate': 0.03}

In [121]:
make_model = tuning.best_estimator_
pred = make_model.predict(test_x)
ans = pd.DataFrame(pred, columns=train_y.columns)
ans.to_csv('C:/Users/gaman/Desktop/LG Aimers/해커톤/final3.csv', index=False)

In [105]:
# train_score = pd.DataFrame(columns={'score'}) # train_x 스코어 저장할 dataframe
submit = pd.read_csv('C:/Users/gaman/Desktop/LG Aimers/해커톤/data/sample_submission.csv') # 제출 파일 양식 불러오기



for a in range(1,15): # Y_01 ~ Y_14 반복
  
  # pycaret 형식에 맞는 dataframe 만들기
  if a < 10:
    df = pd.concat([train_x,train_y['Y_0{}'.format(a)]],axis=1)

    # pycaret 모델 만들기
    reg = setup(data = df, target = 'Y_0{}'.format(a), 
                                      silent = True, # enter 생략
                                      session_id = 42,
                                      # fold_shuffle=True,
                                      # imputation_type='iterative',
                                      fold = 10,
                                      # use_gpu = True,
                                      ignore_features = ['X_23','X_47','X_48'], # 제외할 변수
                                      numeric_features= list(train_x.drop(columns = ['X_04','X_23','X_47','X_48']).columns), # 수치형 변수
                                      categorical_features = ['X_04'] ) # 범주형 변수
    

  else:
    df = pd.concat([train_x,train_y['Y_{}'.format(a)]],axis=1)

    # pycaret 모델 만들기
    reg = setup(data = df, target = 'Y_{}'.format(a), 
                                      silent = True, # enter 생략
                                      session_id = 42,
                                      # fold_shuffle=True,
                                      # imputation_type='iterative',
                                      fold = 10,
                                      # use_gpu = True,
                                      ignore_features = ['X_23','X_47','X_48'], # 제외할 변수
                                      numeric_features= list(train_x.drop(columns = ['X_04','X_23','X_47','X_48']).columns), # 수치형 변수
                                      categorical_features = ['X_04'] ) # 범주형 변수
    
    
  
  # RMSE 기준으로 모델 비교해서 예측모델 만들기 (앙상블이나 모델 여러개 합치는 것도 다 가능!)
  # 모델 생성
  
  # top3 = compare_models(sort = 'RMSE', n_select = 3)
  # lgbm = create_model('lightgbm', fold=5)
  # gbr = create_model('gbr', fold =5)
  # rf = create_model('rf', fold =5)
  # catb = create_model('catboost', fold =5)
  # xgb = create_model('xgboost', fold =5)

  # blend models
  print("모델 비교(18m) 및 앙상블 시작")
  blended = blend_models(compare_models(sort = 'RMSE', n_select = 3, exclude = ['rf']), fold = 5, choose_better = True) # 앙상블   

  # finalize model
  model = finalize_model(blended)                    
  
  # # train_x 예측 결과 확인, 평가
  # prediction_train  = predict_model(blended, data = train_x) # train_x 예측. label 생김
  
  # if a < 10:
  #   score = check_metric(train_y['Y_0{}'.format(a)], prediction_train['Label'], metric = 'RMSE')

  # else:
  #   score = check_metric(train_y['Y_{}'.format(a)], prediction_train['Label'], metric = 'RMSE')

  
  # # 데이터프레임에 socre 값 저장
  # train_score.loc[a] = score

  # train_score.to_csv('C:/Users/gaman/Desktop/LG Aimers/해커톤/검사변수 삭제_61x_60~64.csv', index=False)


  # test_x 예측 결과 저장
  print("예측 시작")
  prediction_test = predict_model(model, data = test_x) # test_x 예측, label 생김.

  if a < 10:
    submit['Y_0{}'.format(a)] = prediction_test['Label']

  else:
    submit['Y_{}'.format(a)] = prediction_test['Label']

  
  submit.to_csv('C:/Users/gaman/Desktop/LG Aimers/해커톤/final.csv', index=False)
  print(a)
  
  

# 소요 시간 기록하기!!!!!!!!!!!!!!
# (제발)_ 527m
# 543
# 615
# 508

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4864,0.3953,0.6287,0.0994,0.0230,0.0185
1,0.4842,0.3856,0.6209,0.0893,0.0227,0.0184
2,0.4855,0.3900,0.6245,0.0815,0.0228,0.0184
3,0.4862,0.3919,0.6260,0.0862,0.0229,0.0185
4,0.4862,0.3896,0.6242,0.0924,0.0228,0.0185
Mean,0.4857,0.3905,0.6249,0.0898,0.0228,0.0185
Std,0.0008,0.0032,0.0025,0.0060,0.0001,0.0000


예측 시작
14


# 평가

In [117]:
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

In [123]:
bestt = pd.read_csv(r'C:\Users\gaman\Downloads\(제발)검사변수 삭제_61x_60~64_vscode.csv').drop(columns=['ID'])
test_please = pd.read_csv('C:/Users/gaman/Desktop/LG Aimers/해커톤/final2.csv').drop(columns=['ID'])

new = pd.read_csv(r'C:\Users\gaman\Downloads\final.csv').drop(columns=['ID'])


In [124]:
score_preview_bestt = lg_nrmse(test_y, bestt)
score_preview_new = lg_nrmse(test_y, new)
score_preview_test = lg_nrmse(test_y, test_please)

print('best score : ', score_preview_bestt)
print('neww score : ', score_preview_new)
print('test score : ', score_preview_test)

best score :  2.0132327681063247
neww score :  2.0135890333441506
test score :  2.0236478873220114


In [20]:
score_preview_bestt = lg_nrmse(test_y, bestt)
score_preview_test = lg_nrmse(test_y, test_please)

print('best score : ', score_preview_bestt)
print('test score : ', score_preview_test)

10 fold score :  2.0154989854480903
4, 48변수 삭제 score :  2.0176216429494427
